In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163690 sha256=be940942bd53b23e35b1143fd2902fe43de17614b00034ebb1ffb235f5ca5f6e
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
dataset = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [5]:
trainset, testset = train_test_split(dataset, test_size=.25, random_state=0)

In [6]:
algo = SVD(random_state=0)
algo.fit(trainset)

In [8]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 3개 추출')
predictions[:3]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 3개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False})]

In [9]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [10]:
# predcit method needs uid, iid and r_ui is option
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [11]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

surprise module

In [12]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
ratings.to_csv('ratings_noh.csv', index=False, header=False)

In [14]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./ratings_noh.csv',reader=reader)

In [17]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [18]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

In [19]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [20]:
from surprise.model_selection import cross_validate
ratings = pd.read_csv('./ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [21]:
algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8687  0.8754  0.8755  0.8659  0.8754  0.8722  0.0041  
MAE (testset)     0.6706  0.6721  0.6699  0.6641  0.6733  0.6700  0.0032  
Fit time          2.01    1.53    1.55    1.52    1.59    1.64    0.19    
Test time         0.13    0.32    0.13    0.30    0.13    0.20    0.09    


{'test_rmse': array([0.86873245, 0.87539574, 0.87545723, 0.86593606, 0.87535211]),
 'test_mae': array([0.67055456, 0.67213486, 0.66986927, 0.66406784, 0.67330299]),
 'fit_time': (2.010181427001953,
  1.5321803092956543,
  1.5513637065887451,
  1.5230140686035156,
  1.5935654640197754),
 'test_time': (0.12735486030578613,
  0.3216862678527832,
  0.13153934478759766,
  0.2964646816253662,
  0.12601065635681152)}

In [22]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8769706507388862
{'n_epochs': 20, 'n_factors': 50}


## 개인 영화 평가 시스템

In [24]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

# 100% trainset build.
trainset = data_folds.build_full_trainset()

In [25]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [26]:
movies = pd.read_csv('./movies.csv')


In [27]:
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])
# 38만 존재함

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [28]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)
# 예상 추천 점수 3.13

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [29]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()

    total_movies = movies['movieId'].tolist()

    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [37]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
   predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
   def sortkey_est(pred):
      return pred.est
   predictions.sort(key=sortkey_est, reverse=True)
   top_predictions= predictions[:top_n]
   top_movie_ids = [ int(pred.iid) for pred in top_predictions]
   top_movie_rating = [ pred.est for pred in top_predictions]
   top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
   top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

   return top_movie_preds

In [38]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
  # name, score
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
